# Aborto: El plenario de Comisiones y el Debate en Diputados. Un análisis automático

## El corpus

El corpus está dividido en dos:


1. Las versiones taquigráficas del plenario de Comisiones que tuvo lugar en la Cámara de Diputados todos los martes y jueves entre el 10 de abril y el 24 de mayo de 2018.
2. Las versiones taquigráficas del debate previo a la votación de la ley de interrupción voluntaria del emarbarazo en la Cámara de Diputados.


## Preprocesamiento


En las dos partes del corpus se usaron Expresiones Regulares para captar los patrones que dividían, en los documentos taquigráficos, las líneas de introducción del discurso de cada uno de los expositores/diputados.


Esa extracción se bajó a una tabla que indica el nombre del orador, su profesión (no en todos los casos) y su posición a favor o en contra de la ley del aborto. RegEx también fue la estrategia que se usó para conectar los apellidos de los oradores con estos datos que residían en una tabla diferente (ReGex y mucha buena voluntad de chequear a mano que estuvieran bien, y sin embargo...).


Finalmente, se extrajeron símbolos que podrían perjudicar el procesamiento posterior. Se trabajó únicamente con los discursos que estaban marcados con las ‘etiquetas’ ‘a favor’ y ‘en contra’, es decir, se dejóaron de lado las intervenciones de los presidentes/moderadores del debate.


## Lematización -


Con la lematización se buscó reducir la dimensionalidad del vocabulario sin afectar a las formas completas, para que, al final del proceso, el output fuera legible. Al menos, esa fue la idea inicial. 


Primero, se utilizó un lematizador muy simple que consistía en una lista de, aproximadamente, quinientos mil pares *forma de superficie-lema* separados por una tabulación. 


Esta forma de diccionario presenta varios problemas. Por ejemplo, el diccionario disponible no poseía tildes, por lo que se procedió a eliminar las tildes del corpus, eliminando así la diferencia evidente entre palabras con y sin tilde. A su vez, se debía confiar un poco 'a ciegas' en que las listas fueran lo suficientemente completas y consistentes para que el resultado fuera útil. Por último, una misma forma de superficie podía ser entrada de más de una forma *lema* del diccionario. Pongamos el caso de la palabra ‘estado’, con importancia propia dentro de este corpus, que era tomada como forma verbal y llevada al lema ‘estar’. Al momento de lematizar de forma automática, la decisión de a qué forma llevar cada palabra debía hacerse de ante mano. Se dicidió que, por default, cayera en la primera forma lema. Para el caso específico de 'estado', se evitó la lematización (y consigo, la de las formas verbales reales). Conclusión, esta lematización reducía la dimensionalidad, pero a costa de obtener un vocabulario que no era fiel al corpus original.


Sin embargo, no se abandonó la intención perseguida con el lematizador, y se pocedió al uso de otro que, con el uso de un PoS Tagger de Stanford provisto por la librería NLTK, entrena en el corpus para decidir qué etiqueta sintáctica asignarle a cada palabra y, en base a esa etiqueta, aplica el diccionario de lemas, consiguiendo un vocabulario reducido y fiel (o al menos, mucho más fiel) al original. En este caso, 'el estado' recibiría un POS 


## Extracción de tópicos con LDA - la lupa sobre el plenario de Comisiones


La hipótesis perseguida en la extracción de tópicos era la idea de que cada postura frente al aborto (‘a favor’ y ‘en contra’) utilizaría distintos tópicos para apoyar su discurso y contradecir el opuesto. Estos tópicos incluirían distintas maneras (y,  por lo tanto, palabras) de tratar el tema desde lo social, económico, político, científico, médico, etc. Entonces, un análisis de estos tópicos podría ser de gran utilidad en una tarea de clasificación.


Para explorar esta hipótesis, se aplicó LDA en el total de los textos y en los textos separados por la etiqueta de ‘posición’. 

### Las tres visualizaciones

#### General
En [LDAgeneral](./LDAgeneral.html#topic=0&lambda=1&term=) se visualizan diez tópicos extraídos de todo el corpus. 
La idea detrás de esta visualización es que, en cierta medida, debería reflejar las dos posiciones encontradas en los discursos del plenario de Comisiones. Es decir, si la hipótesis de que cada 'bando' utiliza sus propios argumentos respaldados con una 'bolsa de palabras' particular (saltando el hecho de que los dos hablan del mismo tema general), filtrando las palabras de mayor frecuencia, deberíamos poder ver cómo estos tópicos agrupan esos subtemas. 


En el gráfico de la izquierda, vemos que, de diez tópicos (el número final de tópicos varió entre los 5 y los 20, quedando 10 como decisión final), tres son los que tienen mayor predominancia en el modelo [<sup>1</sup>](#footnote1) (1, 2 y 3). Entre sí, 2 y 3 son los que más se alejan o tienen menor relación entre ellos. 


##### Un doble click sobre los tópicos 1 y 2: parámetro λ


A 'ojo', el tópico 1 parece representar más a las posturas que están 'a favor' y el tópico 2, a las que están 'en contra'.

El panel de la derecha muestra la relevancia de las palabras dado un tópico. Si miramos las del tópico 1, podemos ver que las de mayor relevancia dentro de ese tópico incluyen palabras como **salud**, **legal**, **seguro**, **social** y **acceso**. Entre las palabras más relevantes del tópico 2 se encuentran **persona**, **niño**, **concepción**, **nacer** y **protección**.

Sin embargo, los resultados más interesantes para estos tópicos aparecen al bajar el parámetro **λ** de la métrica de relevancia porque revela palabras con fuerte contenido en uno y otro de estos dos tópicos principales, como **animal**, **inconstitucional** o **razonamiento** para el tópico 2 y **lesbiana**, **mito** y **antiderecho** para el tópico 1.

Una vez explorada la visión general de los tópicos y haciendo uso de la etiqueta que diferencia a los discursos, se procedió a trabajar del mismo modo con tópicos internos a los corpus de cada postura, a modo de exploración sobre qué temas tocan una y otra. 

Para estos casos también se eligió un modelo de 10 tópicos.

#### Postura *a favor* de la despenalización del aborto

* El tópico 1 une palabras relacionadas con la religión: 'iglesia', 'creyente', 'laico'. 
* El tópico 2 aborda aspectos prácticos del debate: 'acceso', 'hospital, 'practicar'.
* El tópico 3 se acerca a las palabras que son 'caballito de batalla' en los discursos de esta postura: 'desición', 'clandestino', 'cuerpo', 'morir'.


[LDAafavor](./LDAafavor.html)

#### Postura *en contra* de la despenalización del aborto

El tópico 1 tiene un sesgo orientado a la legislación: 'ley', 'proyecto', 'derecho'.
El tópico 2 va hacia la perspectiva biológica: 'embrión', 'celulas', 'genética'.
El tópico 3 vuelve al derecho pero con una visión internacional ¿?: 'interamericana', 'san' + 'josé', 'tratado'.

[LDAencontra](./LDAcontra_sinfiltrar.html#topic=0&lambda=1&term=)

### Conclusiones

Con el trabajo sobre el vocabulario en la lematización y con el ajuste de los parámetros propios del método LDA, este análisis parece conveniente no sólo como trabajo exploratorio de los discursos (¿de qué se habla cuando se está a favor del aborto? ¿y cuándo se está en contra?), sino también como feature para entrenar un clasificador.



<a id='footnote1'>1.</a> Sievert, C & Shirley, K.E.. (2014). LDAvis: A method for visualizing and interpreting topics. Proceedings of the Workshop on Interactive Language Learning, Visualization, and Interfaces. 63-70. 